In [2]:
import pandas as pd
import psycopg2
from psycopg2 import sql

# Conectar ao banco de dados
conn = psycopg2.connect(
    dbname='defaultdb',
    user='avnadmin',
    password='AVNS_R_8PgcoiAljts5A2h1U',
    host='trabalhobd2-jhonpinhosantos-75df.e.aivencloud.com',
    port='23090',
    sslmode='require'
)
cur = conn.cursor()

# Função para executar consulta e retornar DataFrame
def executar_consulta(consulta):
    try:
        cur.execute(consulta)
        colnames = [desc[0] for desc in cur.description]
        data = cur.fetchall()
        df = pd.DataFrame(data, columns=colnames)
        print(f"Consulta: {consulta}\n")
        print(df)
        print("\n" + "="*40 + "\n")
    except Exception as e:
        print(f"Erro na consulta: {consulta}")
        print(f"Erro: {e}")
        conn.rollback()  # Rollback em caso de erro

# Consultas 9.1 - Tabelas com todos os dados inseridos
consultas_tabelas = [
    "SELECT * FROM ORDEM_SERVICO;",
    "SELECT * FROM PEDIDO;",
    "SELECT * FROM CATALOGO;",
    "SELECT * FROM ORCAMENTO;",
    "SELECT * FROM ORDEM_COMPRA;",
    "SELECT * FROM ITENS_PEDIDO;"
]

print("9.1 Consultas das tabelas com todos os dados inseridos:")
for consulta in consultas_tabelas:
    executar_consulta(consulta)

# Consultas 9.2 - Filtros WHERE
consultas_where = [
    "SELECT * FROM ORDEM_SERVICO WHERE id_equipamento > 1003;",
    "SELECT * FROM PEDIDO WHERE data_pedido BETWEEN '2023-01-01' AND '2023-06-30';",
    "SELECT * FROM CATALOGO WHERE tipo = 'Serviço';",
    "SELECT * FROM ORCAMENTO WHERE fornecedor LIKE 'Fornecedor A';"
]

print("9.2 Consultas das tabelas com filtros WHERE:")
for consulta in consultas_where:
    executar_consulta(consulta)

# Consultas 9.3 - Operadores lógicos, aritméticos e renomeação de campos

# a) Consultas com operadores lógicos (AND, OR, NOT)
consultas_logicas = [
    "SELECT * FROM ORDEM_SERVICO WHERE data_inicio < '2024-01-01' AND data_conclusao > '2023-12-01';",
    "SELECT * FROM PEDIDO WHERE data_recebimento >= '2023-01-01' OR data_recebimento <= '2023-06-30';",
    "SELECT * FROM CATALOGO WHERE tipo = 'Peça' AND descricao NOT LIKE 'Filtro%';",
    "SELECT * FROM ORCAMENTO WHERE fornecedor = 'Fornecedor B' OR fornecedor = 'Fornecedor C';",
    "SELECT * FROM ORDEM_COMPRA WHERE FK_ORCAMENTO_id_orcamento IS NOT NULL;"
]

# b) Consultas com operadores aritméticos
consultas_aritmeticas = [
    "SELECT FK_PEDIDO_id_pedido, valor, quantidade, valor * quantidade AS total FROM ITENS_PEDIDO;",
    "SELECT FK_PEDIDO_id_pedido, valor, garantia, valor / garantia AS valor_por_mes FROM ITENS_PEDIDO;",
    "SELECT FK_PEDIDO_id_pedido, quantidade, quantidade * 2 AS quantidade_dobrada FROM ITENS_PEDIDO;"
]

# c) Consultas com renomeação de campos ou tabelas
consultas_renomeacao = [
    "SELECT id_pedido AS pedido_id, data_pedido AS pedido_data FROM PEDIDO;",
    "SELECT descricao AS item_descricao, tipo AS item_tipo FROM CATALOGO;",
    "SELECT fornecedor AS fornecedor_nome FROM ORCAMENTO;"
]

print("9.3 Consultas com operadores lógicos, aritméticos e renomeação de campos:")
for consulta in consultas_logicas + consultas_aritmeticas + consultas_renomeacao:
    executar_consulta(consulta)

# Consultas 9.4 - Operadores LIKE e funções de data

# a) Consultas com LIKE e ILIKE
consultas_like = [
    "SELECT * FROM CATALOGO WHERE descricao LIKE 'Pneu%';",
    "SELECT * FROM CATALOGO WHERE descricao ILIKE 'serviço%';",
    "SELECT * FROM ORCAMENTO WHERE fornecedor LIKE 'Fornecedor%';",
    "SELECT * FROM PEDIDO WHERE TO_CHAR(data_pedido, 'YYYY-MM') LIKE '2023-01%';",
    "SELECT * FROM ORDEM_SERVICO WHERE TO_CHAR(data_conclusao, 'YYYY-MM') LIKE '2023-08%';"
]

# b) Consultas com funções de data
consultas_data = [
    "SELECT *, CURRENT_DATE AS data_atual FROM ORDEM_SERVICO;",
    "SELECT *, EXTRACT(YEAR FROM data_pedido) AS ano_pedido FROM PEDIDO;",
    "SELECT *, DATE_PART('year', data_inicio) AS ano_inicio FROM ORDEM_SERVICO;"
]

print("9.4 Consultas com operadores LIKE e funções de data:")
for consulta in consultas_like + consultas_data:
    executar_consulta(consulta)

# Consultas 9.5 - Atualização e Exclusão de Dados

# a) Consultas de exclusão
consultas_exclusao = [
    "DELETE FROM ORDEM_SERVICO WHERE id_os = 15;",
    "DELETE FROM PEDIDO WHERE id_pedido = 15;",
    "DELETE FROM CATALOGO WHERE id_item = 15;"
]

# b) Consultas de atualização
consultas_atualizacao = [
    "UPDATE PEDIDO SET data_recebimento = '2023-07-01' WHERE id_pedido = 1;",
    "UPDATE CATALOGO SET tipo = 'Peça' WHERE tipo IS NULL;",
    "UPDATE ORCAMENTO SET fornecedor = 'Fornecedor X' WHERE id_orcamento = 1;"
]

print("9.5 Consultas com instruções de atualização e exclusão de dados:")
for consulta in consultas_exclusao + consultas_atualizacao:
    executar_consulta(consulta)


# Consultas 9.6 - INNER JOIN e ORDER BY

# a) Junção envolvendo todas as tabelas
consulta_inner_join_todas = """
    SELECT o.id_os, p.id_pedido, c.descricao, i.valor
    FROM ORDEM_SERVICO o
    INNER JOIN PEDIDO p ON o.id_os = p.FK_ORDEM_SERVICO_id_os
    INNER JOIN ITENS_PEDIDO i ON p.id_pedido = i.FK_PEDIDO_id_pedido
    INNER JOIN CATALOGO c ON i.FK_ITENS_id_item = c.id_item
    ORDER BY o.id_os, p.id_pedido;
"""

# b) Outras junções importantes
consultas_inner_join = [
    """
    SELECT p.id_pedido, o.data_inicio, i.valor
    FROM PEDIDO p
    INNER JOIN ORDEM_SERVICO o ON p.FK_ORDEM_SERVICO_id_os = o.id_os
    INNER JOIN ITENS_PEDIDO i ON p.id_pedido = i.FK_PEDIDO_id_pedido
    ORDER BY p.id_pedido DESC;
    """,
    """
    SELECT c.descricao, o.id_os, COUNT(p.id_pedido) AS total_pedidos
    FROM CATALOGO c
    INNER JOIN ITENS_PEDIDO i ON c.id_item = i.FK_ITENS_id_item
    INNER JOIN PEDIDO p ON i.FK_PEDIDO_id_pedido = p.id_pedido
    INNER JOIN ORDEM_SERVICO o ON p.FK_ORDEM_SERVICO_id_os = o.id_os
    GROUP BY c.descricao, o.id_os
    ORDER BY total_pedidos DESC;
    """
]

print("9.6 Consultas com INNER JOIN e ORDER BY:")
executar_consulta(consulta_inner_join_todas)
for consulta in consultas_inner_join:
    executar_consulta(consulta)

# Consultas 9.7 - GROUP BY e funções de agrupamento

# a) Consultas com junção
consultas_group_by = [
    """
    SELECT o.id_os, COUNT(p.id_pedido) AS total_pedidos
    FROM ORDEM_SERVICO o
    LEFT JOIN PEDIDO p ON o.id_os = p.FK_ORDEM_SERVICO_id_os
    GROUP BY o.id_os;
    """,
    """
    SELECT c.descricao, SUM(i.quantidade) AS total_quantidade
    FROM CATALOGO c
    INNER JOIN ITENS_PEDIDO i ON c.id_item = i.FK_ITENS_id_item
    GROUP BY c.descricao
    ORDER BY total_quantidade DESC;
    """
]

print("9.7 Consultas com GROUP BY e funções de agrupamento:")
for consulta in consultas_group_by:
    executar_consulta(consulta)

# Consultas 9.8 - LEFT, RIGHT e FULL JOIN

# a) LEFT JOIN
consulta_left_join = """
    SELECT p.id_pedido, o.id_os
    FROM PEDIDO p
    LEFT JOIN ORDEM_SERVICO o ON p.FK_ORDEM_SERVICO_id_os = o.id_os;
"""

# b) RIGHT JOIN
consulta_right_join = """
    SELECT p.id_pedido, o.id_os
    FROM PEDIDO p
    RIGHT JOIN ORDEM_SERVICO o ON p.FK_ORDEM_SERVICO_id_os = o.id_os;
"""

# c) FULL JOIN
consulta_full_join = """
    SELECT p.id_pedido, o.id_os
    FROM PEDIDO p
    FULL JOIN ORDEM_SERVICO o ON p.FK_ORDEM_SERVICO_id_os = o.id_os;
"""

print("9.8 Consultas com LEFT, RIGHT e FULL JOIN:")
executar_consulta(consulta_left_join)
executar_consulta(consulta_right_join)
executar_consulta(consulta_full_join)

# Consultas 9.9 - SELF JOIN e VIEW

# a) SELF JOIN
consulta_self_join = """
    SELECT a.id_pedido AS pedido_id_1, b.id_pedido AS pedido_id_2
    FROM PEDIDO a
    INNER JOIN PEDIDO b ON a.FK_ORDEM_SERVICO_id_os = b.FK_ORDEM_SERVICO_id_os;
"""

# b) VIEW
consulta_create_view = """
    CREATE VIEW vw_pedido_info AS
    SELECT p.id_pedido, o.data_inicio, o.data_conclusao, c.descricao
    FROM PEDIDO p
    INNER JOIN ORDEM_SERVICO o ON p.FK_ORDEM_SERVICO_id_os = o.id_os
    INNER JOIN ITENS_PEDIDO i ON p.id_pedido = i.FK_PEDIDO_id_pedido
    INNER JOIN CATALOGO c ON i.FK_ITENS_id_item = c.id_item;
"""

consulta_use_view = "SELECT * FROM vw_pedido_info;"

print("9.9 Consultas com SELF JOIN e VIEW:")
executar_consulta(consulta_self_join)

executar_consulta(consulta_use_view)

# Consultas 9.10 - Subconsultas

# a) Subconsulta com GROUP BY
subconsulta_group_by = """
    SELECT id_os, (SELECT COUNT(*) FROM PEDIDO WHERE FK_ORDEM_SERVICO_id_os = o.id_os) AS total_pedidos
    FROM ORDEM_SERVICO o
    GROUP BY id_os;
"""

# b) Subconsulta com junção
subconsulta_juncao = """
    SELECT p.id_pedido, (SELECT c.descricao FROM CATALOGO c WHERE c.id_item = i.FK_ITENS_id_item) AS item_descricao
    FROM PEDIDO p
    INNER JOIN ITENS_PEDIDO i ON p.id_pedido = i.FK_PEDIDO_id_pedido;
"""

print("9.10 Subconsultas:")
executar_consulta(subconsulta_group_by)
executar_consulta(subconsulta_juncao)

# Fechar conexão
cur.close()
conn.close()


9.1 Consultas das tabelas com todos os dados inseridos:
Consulta: SELECT * FROM ORDEM_SERVICO;

    id_os  id_equipamento data_inicio data_conclusao
0       1            1001  2023-01-01     2023-01-10
1       2            1002  2023-02-01     2023-02-15
2       3            1003  2023-03-01     2023-03-10
3       4            1004  2023-04-01     2023-04-15
4       5            1005  2023-05-01     2023-05-15
5       6            1006  2023-06-01     2023-06-15
6       7            1007  2023-07-01     2023-07-15
7       8            1008  2023-08-01     2023-08-15
8       9            1009  2023-09-01     2023-09-15
9      10            1010  2023-10-01     2023-10-15
10     11            1011  2023-11-01     2023-11-15
11     12            1012  2023-12-01     2023-12-15
12     13            1013  2024-01-01     2024-01-15
13     14            1014  2024-02-01     2024-02-15
14     15            1015  2024-03-01     2024-03-15
15     16            1016  2024-04-01     2024-04-15
16 